In [ ]:
from datetime import datetime

from dotenv import load_dotenv
from haversine import haversine, Unit
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import seaborn as sns
import os
import pandas as pd

In [ ]:
load_dotenv('./env.env')

driver_name = 'postgresql+psycopg2'
url = URL.create(
    drivername=driver_name,
    username=os.environ['USERNAME'],
    password=os.environ['PASSWORD'],
    host=os.environ['HOST'],
    port=os.environ['PORT'],
    database=os.environ['DB']
)
engine = create_engine(url)
db_connection = engine.connect()

In [ ]:
query = """
select h.id as event_id, h.facility_uuid, v.uid as visit_facility_uuid, h."timestamp", h.latitude as event_latitude,
h.longitude as event_longitude, h."event", h.altitude,
v.project_id, v.latitude as facility_latitude, v.longitude as facility_longitude, v.revision,
v.inspection_document, v.person_of_interest_id
from custom h join visit v on v.uid = h.facility_uuid join usr u on v.person_of_interest_id = u.id where h."event" = 9 or h."event" = 13 order by h."timestamp" desc
"""

df = pd.read_sql(query, db_connection)

In [ ]:
len(df)

In [ ]:
df_per_facility_uuid = [y for _, y in df.groupby('facility_uuid')]
print(len(df_per_facility_uuid))


def get_datetime(date_time: str) -> datetime:
    index = date_time.rfind(' ')
    date_time = date_time[:index] + date_time[index + 1:]
    return datetime.fromisoformat(date_time)


def get_elapsed_time(t1: str, t2: str) -> int:
    datetime_1 = get_datetime(t1)
    datetime_2 = get_datetime(t2)
    return (datetime_1 - datetime_2).seconds


start_time = str()

for df_ in df_per_facility_uuid:
    number_of_save_and_exit_counts = []
    elapsed_times = []
    distances = []
    number_of_save_and_exit_count = 0

    for i in df_.sort_values(by='timestamp').index:
        if df_['event'][i] == 13:
            start_time = df_['timestamp'][i]
            elapsed_times.append(0)
            distances.append(0)
            number_of_save_and_exit_counts.append(0)
        elif df_['event'][i] == 9:
            number_of_save_and_exit_count += 1
            number_of_save_and_exit_counts.append(number_of_save_and_exit_count)

            end_time = df_['timestamp'][i]
            elapsed_times.append((end_time - start_time).seconds)

            event_lat = df_['event_latitude'][i]
            event_long = df_['event_longitude'][i]
            event = (event_lat, event_long)

            fac_lat = df_['facility_latitude'][i]
            fac_long = df_['facility_longitude'][i]
            facility = (fac_lat, fac_long)

            distances.append(haversine(event, facility, Unit.METERS))

    elapsed_times.reverse()
    distances.reverse()
    number_of_save_and_exit_counts.reverse()
    df_['elapsed_time'] = elapsed_times
    df_['distance'] = distances
    df_['save_and_exit_count'] = number_of_save_and_exit_counts
    #print(df_[['event', 'timestamp', 'elapsed_time', 'distance']])

print(df_per_facility_uuid[0][['event', 'timestamp', 'elapsed_time', 'distance', 'save_and_exit_count']])

In [ ]:
df = pd.concat(df_per_facility_uuid, ignore_index=True)
#print(df[df['event'] == 9][['event', 'timestamp', 'elapsed_time', 'distance']])
df.info()

In [ ]:
#final_joined = pd.concat((df, df_inpsection_documents), axis=1)
#print(len(final_joined))

#df_save_and_exit_only = final_joined[final_joined['event'] == 9]
df_save_and_exit_only = df[df['event'] == 9]
data = df_save_and_exit_only[['elapsed_time', 'distance', 'revision', 'save_and_exit_count']]
print(len(data))


In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt

corr = data.corr()

f, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(corr, cmap="Blues", annot=True, square=False, ax=ax)
plt.title('Pearson Correlation of Features')
plt.yticks(rotation=45)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from pyod.models.iforest import IForest


# Create a model with 10000 trees
iforest = IForest(n_estimators=1000)
iforest.fit(data)  # This will take a minute

# Extract the labels
labels = iforest.labels_

In [ ]:
len(labels)

In [ ]:
X_outlier_free = df_save_and_exit_only[labels == 0]
X_outlier = df_save_and_exit_only[labels == 1]

print(len(X_outlier_free))
print(len(data))
print(len(df_save_and_exit_only))

In [ ]:
feature_importance_mapped = pd.Series({column: importance for column, importance in zip(data.columns, iforest.feature_importances_)})
print(feature_importance_mapped.sort_values(ascending=False))

In [ ]:
df_save_and_exit_only['score'] = iforest.decision_scores_
print('Threshold Value: %s' % iforest.threshold_)
sns.displot(df_save_and_exit_only, x='score', kde=True)

In [ ]:
scores_median = df_save_and_exit_only['score'].median()
percentile_90 = df_save_and_exit_only['score'].quantile(0.9)
percentile_99 = df_save_and_exit_only['score'].quantile(.99)

print('Median Score: %s' % scores_median)
print('90th Percentile Score: %s' % percentile_90)

lower_fifty = df_save_and_exit_only[df_save_and_exit_only.score<scores_median].sort_values(by='score', ascending=False)
upper_fifty = df_save_and_exit_only[df_save_and_exit_only.score>=scores_median].sort_values(by='score', ascending=False)
upper_90 = df_save_and_exit_only[df_save_and_exit_only.score>=percentile_90].sort_values(by='score', ascending=False)
upper_99 = df_save_and_exit_only[df_save_and_exit_only.score>=percentile_99].sort_values(by='score', ascending=False)


In [ ]:
example_50th_lower = lower_fifty.iloc[0]
example_50th_upper = upper_fifty.iloc[-1]
example_90th_highest = upper_90.iloc[0]
example_90th_lowest = upper_90.iloc[-1]

print('Example Lower 50th -- Facility UUID: %s, Score: %s' % (example_50th_lower['facility_uuid'], example_50th_lower['score']))
print('Example Upper 50th -- Facility UUID: %s, Score: %s' % (example_50th_upper['facility_uuid'], example_50th_upper['score']))
print('Example Lowest 90th -- Facility UUID: %s, Score: %s' % (example_90th_lowest['facility_uuid'], example_90th_lowest['score']))
print('Example Highest 90th -- Facility UUID: %s, Score: %s' % (example_90th_highest['facility_uuid'], example_90th_highest['score']))


In [ ]:
#print(df[df.scores > 0.1].head())
print(data.elapsed_time.median())
print(data.distance.median())

In [ ]:
#sns.displot(data, x='elapsed_time')
elapsed_times_boxplot = sns.boxplot(data, x='elapsed_time')
data[data['elapsed_time'] > 60000]

In [ ]:
sns.boxplot(data, x='distance')
data[data['distance'] > 60000]

In [ ]:
print(len(upper_99))
upper_99.head()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=20, contamination='auto')
y_pred = clf.fit_predict(data)
#X_scores = clf.negative_outlier_factor_
X_scores = iforest.decision_scores_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerPathCollection

 
# explicit function to normalize array
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)    
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return np.array(norm_arr)


def update_legend_marker_size(handle, orig):
    "Customize size of the legend marker"
    handle.update_from(orig)
    handle.set_sizes([20])


plt.scatter(data['elapsed_time'].values, data['distance'].values, color="k", s=3.0, label="Data points")
# plot circles with radius proportional to the outlier scores
normalized = normalize(X_scores, 0, 1)
radius = (normalized.max() - normalized) / (normalized.max() - normalized.min())
scatter = plt.scatter(
    data['elapsed_time'].values,
    data['distance'].values,
    s=100,
    c=X_scores,
    edgecolors="r",
    facecolors="none",
    label="Outlier scores",
)
plt.axis("tight")
#plt.xlim((-5, 5))
#plt.ylim((-5, 5))
plt.xlabel("Elapsed Time (s)")
plt.ylabel('Distance (m)')
plt.legend(
    handler_map={scatter: HandlerPathCollection(update_func=update_legend_marker_size)}
)
plt.title("Local Outlier Factor (LOF)")
plt.colorbar().set_label('Outlier Score', rotation=270)
plt.show()

In [ ]:
dfs_grouped_by_person_of_interest = [y for _, y in df_save_and_exit_only.groupby('person_of_interest_id')]
dfs_grouped_by_person_of_interest[0].sort_values(by='timestamp', ascending=False, inplace=True)
g = dfs_grouped_by_person_of_interest[0].rolling(window='7d', closed='both', on='timestamp').distance.mean()

In [ ]:
print(g.values)


In [ ]:
g.values[0]

In [ ]:
dfs_grouped_by_person_of_interest[0]['mean_distance'] = g.values
dfs_grouped_by_person_of_interest[0][['timestamp', 'distance', 'mean_distance']].head(20)

In [ ]:
dfs_grouped_by_person_of_interest = [y for _, y in df_save_and_exit_only.groupby('person_of_interest_id')]
print(len(dfs_grouped_by_person_of_interest))

for df_ in dfs_grouped_by_person_of_interest:
    df_.sort_values(by='timestamp', ascending=False, inplace=True)
    window = df_.rolling(window='7d', on='timestamp', closed='both')
    mean_distance = window.distance.mean()
    mean_elapsed_time = window.elapsed_time.mean()
    mean_revision = window.revision.mean()
    mean_save_and_exit_count = window.save_and_exit_count.mean()

    df_['mean_distance'] = mean_distance.values
    df_['mean_elapsed_time'] = mean_elapsed_time.values
    df_['mean_revision'] = mean_revision.values
    df_['mean_save_and_exit_count'] = mean_save_and_exit_count.values

df_with_means = pd.concat(dfs_grouped_by_person_of_interest)
df_with_means.head(20)

In [ ]:
import matplotlib.pyplot as plt

data = df_with_means[['mean_distance', 'mean_elapsed_time', 'mean_revision', 'mean_save_and_exit_count']]
corr = data.corr()

f, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(corr, cmap="Blues", annot=True, square=False, ax=ax)
plt.title('Pearson Correlation of Features')
plt.yticks(rotation=45)

In [ ]:
clf_person_of_interests = IForest()
clf_person_of_interests.fit(data)

# Extract the labels
person_of_interests_labels = clf_person_of_interests.labels_
feature_importance_mapped = pd.Series({column: importance for column, importance in zip(data.columns, clf_person_of_interests.feature_importances_)})
print(feature_importance_mapped.sort_values(ascending=False))


In [ ]:
df_with_means['person_of_interest_score'] = clf_person_of_interests.decision_scores_

print('Threshold Value: %s' % clf_person_of_interests.threshold_)
sns.displot(df_with_means, x='person_of_interest_score', kde=True)